In [270]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "https://stats.oecd.org/index.aspx?DataSetCode=REVGBR"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
soup

table = soup.find("table", {'class': 'DataTable'})

headers = []

# Scraping the whole table didn't work, therefore scraping relevant rows of the table.
# This is the for the dates column

for i in table.find_all('th', {'class': 'HDim'}):
    title = i.text.strip()
    headers.append(title)

df = pd.DataFrame(columns=headers)


df.insert(0, 'Empty', '')
df.insert(0, 'Year', '')

df = df.iloc[:, :59]

pd.set_option('max_columns', None)
print(df.columns.tolist())




['Year', 'Empty', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


In [271]:
# Now scraping the data row and cleaning it by removing the unicode symbol.

for row in table.find_all('tr', {'id': 'row1'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


total_tax = [s.replace('\xa0', '') for s in row_data]
total_tax

['Total tax revenue',
 '',
 '10945',
 '11946',
 '13320',
 '14976',
 '16934',
 '19080',
 '20167',
 '21474',
 '23324',
 '28900',
 '37343',
 '43954',
 '50389',
 '55380',
 '63624',
 '81153',
 '92880',
 '108233',
 '113345',
 '122092',
 '133701',
 '145729',
 '154902',
 '173088',
 '186990',
 '202368',
 '205304',
 '207654',
 '211438',
 '228041',
 '249597',
 '264778',
 '284587',
 '312365',
 '332778',
 '359690',
 '369143',
 '374624',
 '392640',
 '424991',
 '454976',
 '483578',
 '509308',
 '512205',
 '481292',
 '515253',
 '546130',
 '550431',
 '570196',
 '590196',
 '611338',
 '649143',
 '681497',
 '704795',
 '726723',
 '690450',
 '775646']

In [272]:
# Creating a dataframe from the scraped data

df2 = pd.DataFrame([total_tax])
df = df.columns.to_frame().T.append(df, ignore_index=True)
df.columns = range(len(df.columns))
df1=df.append(df2)

df1=df1.reset_index(drop=True)
df1



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,Year,Empty,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,Total tax revenue,,10945,11946,13320,14976,16934,19080,20167,21474,23324,28900,37343,43954,50389,55380,63624,81153,92880,108233,113345,122092,133701,145729,154902,173088,186990,202368,205304,207654,211438,228041,249597,264778,284587,312365,332778,359690,369143,374624,392640,424991,454976,483578,509308,512205,481292,515253,546130,550431,570196,590196,611338,649143,681497,704795,726723,690450,775646


In [273]:
# Same again for row 5

for row in table.find_all('tr', {'id': 'row5'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


income_tax = [s.replace('\xa0', '') for s in row_data]

df3 = pd.DataFrame([income_tax])

df1 = df1.append(df3)
df1[0] = df1[0].str.replace("1110 On income and profits", 'Income tax')
df1 = df1.reset_index(drop=True)

df1[0] = df1[0].str.replace("Year", '')
df1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,,Empty,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,Total tax revenue,,10945,11946,13320,14976,16934,19080,20167,21474,23324,28900,37343,43954,50389,55380,63624,81153,92880,108233,113345,122092,133701,145729,154902,173088,186990,202368,205304,207654,211438,228041,249597,264778,284587,312365,332778,359690,369143,374624,392640,424991,454976,483578,509308,512205,481292,515253,546130,550431,570196,590196,611338,649143,681497,704795,726723,690450,775646
2,Income tax,,3615,4446,4115,4585,5157,5752,6493,6687,7371,9869,14448,16801,17453,18785,20355,23376,26958,29786,31023,32064,33965,39283,40040,43268,46546,57614,58951,61021,60987,65717,71121,71206,72525,85903,93079,101925,107661,108525,110203,117369,127803,136611,147309,149228,138023,144502,149353,146474,152668,156187,163291,169076,177403,184957,191303,186966,218021


In [274]:
# Repeating the process for the required rows.
# A loop may have been possible, though there a quite a few idiosyncrasies 
# It was therefore easier to copy and paste, changing the various fields

for row in table.find_all('tr', {'id': 'row21'}): 
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


National_Insurance = [s.replace('\xa0', '') for s in row_data]

df_NI = pd.DataFrame([National_Insurance])

df1 = df1.append(df_NI)
df1[0] = df1[0].str.replace("2100 Employees", 'Employee National Insurance')
df1 = df1.reset_index(drop=True)



for row in table.find_all('tr', {'id': 'row27'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


National_Insurance_SE = [s.replace('\xa0', '') for s in row_data]

df_NI_SE = pd.DataFrame([National_Insurance_SE])

df1 = df1.append(df_NI_SE)
df1[0] = df1[0].str.replace(
    "2300 Selfemployed or nonemployed", 'Self employed National Insurance')
df1 = df1.reset_index(drop=True)



for row in table.find_all('tr', {'id': 'row8'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


CG = [s.replace('\xa0', '') for s in row_data]

df_CG = pd.DataFrame([CG])

df1 = df1.append(df_CG)
df1[0] = df1[0].str.replace(
    "1120 On capital gains", 'Capital gains')
df1 = df1.reset_index(drop=True)



for row in table.find_all('tr', {'id': 'row9'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


Corp = [s.replace('\xa0', '') for s in row_data]

df_Corp= pd.DataFrame([Corp])

df1 = df1.append(df_Corp)
df1[0] = df1[0].str.replace(
    "1200 Corporate", 'Corporation tax')
df1 = df1.reset_index(drop=True)


for row in table.find_all('tr', {'id': 'row24'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


Employer_NI = [s.replace('\xa0', '') for s in row_data]

df_Employer_NI = pd.DataFrame([Employer_NI])

df1 = df1.append(df_Employer_NI)
df1[0] = df1[0].str.replace(
    "2200 Employers", 'Employers NI')
df1 = df1.reset_index(drop=True)



for row in table.find_all('tr', {'id': 'row34'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


Property_taxes = [s.replace('\xa0', '') for s in row_data]

df_Property_taxes = pd.DataFrame([Property_taxes])

df1 = df1.append(df_Property_taxes)
df1[0] = df1[0].str.replace(
    "4000 Taxes on property", 'Property taxes')
df1 = df1.reset_index(drop=True)




In [275]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# The page is interactive and the table is across mutple pages.
# I therefore use selenium to change table to the next page so it can be scraped.

options = Options()
options.add_argument("start-maximized")

webdriver_service = Service('C:\webdrivers\chromedriver.exe')
driver = webdriver.Chrome(options=options, service=webdriver_service)
wait = WebDriverWait(driver, 5)

url = "https://stats.oecd.org/index.aspx?DataSetCode=REVGBR"
driver.get(url)

select = Select(driver.find_element(By.ID, 'PAGE'))
select.select_by_value('2')


In [276]:
# After selenium has changed the page, I can now scrape the data on page 2 of the table

import time
time.sleep(10)


html = driver.page_source
soup = BeautifulSoup(html)
soup


table = soup.find("table", {'class': 'DataTable'})

for row in table.find_all('tr', {'id': 'row6'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


Inheritance_taxes = [s.replace('\xa0', '') for s in row_data]

df_Inheritance_taxes = pd.DataFrame([Inheritance_taxes])

df1 = df1.append(df_Inheritance_taxes)
df1[0] = df1[0].str.replace(
    "4300 Estate, inheritance and gift taxes", 'Inheritance tax')
df1 = df1.reset_index(drop=True)


for row in table.find_all('tr', {'id': 'row26'}):
    data = row.find_all('td')
    row_data = [td.text.strip() for td in data]


VAT = [s.replace('\xa0', '') for s in row_data]

df_VAT = pd.DataFrame([VAT])



df1 = df1.append(df_VAT)
df1[0] = df1[0].str.replace(
    "5111 Value added taxes", 'VAT')
df1 = df1.reset_index(drop=True)

# All the needed data is now in this dataframe
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,,Empty,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,Total tax revenue,,10945,11946,13320,14976,16934,19080,20167,21474,23324,28900,37343,43954,50389,55380,63624,81153,92880,108233,113345,122092,133701,145729,154902,173088,186990,202368,205304,207654,211438,228041,249597,264778,284587,312365,332778,359690,369143,374624,392640,424991,454976,483578,509308,512205,481292,515253,546130,550431,570196,590196,611338,649143,681497,704795,726723,690450,775646
2,Income tax,,3615,4446,4115,4585,5157,5752,6493,6687,7371,9869,14448,16801,17453,18785,20355,23376,26958,29786,31023,32064,33965,39283,40040,43268,46546,57614,58951,61021,60987,65717,71121,71206,72525,85903,93079,101925,107661,108525,110203,117369,127803,136611,147309,149228,138023,144502,149353,146474,152668,156187,163291,169076,177403,184957,191303,186966,218021
3,Employee National Insurance,,770,808,854,953,992,1171,1237,1468,1693,1991,2562,3114,3520,3690,4284,5228,6552,8016,9413,10200,11220,12625,12513,13564,14140,13296,13828,14104,14748,17357,18646,19149,21558,22528,23573,24175,25236,25357,29055,32805,35159,37052,36585,38186,37184,38703,40626,41159,41481,42556,44488,47295,50397,52203,56222,56847,61073
4,Self employed National Insurance,,75,79,82,94,99,114,120,144,169,193,195,228,277,319,264,317,355,482,552,600,745,725,755,942,1048,1177,1206,1281,1472,1469,1541,1771,1848,1729,1883,2049,2183,2318,2595,2727,2825,2930,2860,3053,2850,2756,2641,2560,2585,2672,3194,3295,3580,3848,4356,4101,4105
5,Capital gains,,3,7,22,51,156,264,245,276,377,496,488,420,403,416,529,492,487,592,643,694,855,971,1230,2172,2002,1892,1359,1073,785,821,852,972,482,1460,1971,2211,3400,2820,1526,2296,2332,2874,3848,5288,7815,2517,3600,4334,3902,3914,5582,7060,8560,7797,9187,10253,12109
6,Corporation tax,,484,141,1171,1287,1358,1658,1527,1429,1939,2847,2310,2168,3037,3839,4884,6783,8471,10492,12069,13997,16854,15345,16264,18808,23433,20070,16736,12976,12578,15398,20174,24726,32001,34020,32743,38061,34255,31274,31467,38076,47014,50269,50159,45856,38823,45752,45216,43222,43199,45026,43824,54611,54561,54691,51253,50077,61360
7,Employers NI,,831,906,976,1101,1136,1354,1453,1708,2054,2791,4068,5066,5694,6069,6889,8210,8814,9344,10536,11269,12245,13540,14395,16176,18145,19984,21159,21621,23038,23240,24210,25579,27200,28731,31286,34028,35706,35735,39890,43692,46475,49568,53765,57080,54411,55887,58174,60600,62019,63892,66491,71373,76472,79327,82728,84042,89367
8,Property taxes,,1591,1742,1875,2074,2265,2378,2679,3162,3166,3686,4746,5363,6057,6641,7902,9774,12386,13850,13981,14603,16007,18101,20393,22356,23676,16538,15374,15284,21194,23808,25033,27218,29641,32543,35988,41134,42247,44644,46555,50393,53838,59656,63392,60086,58413,61768,63405,65320,70889,74593,76427,81212,85545,88654,90329,80134,88393
9,Inheritance tax,,287,308,317,380,450,383,404,483,419,380,307,390,393,372,418,479,1010,701,690,750,923,1025,1089,1107,1182,1321,1263,1227,1279,1438,1441,1621,1601,1795,1951,2215,2396,2381,2416,2871,3150,3575,3867,3257,2401,2642,2936,3129,4255,3886,4470,4804,5381,5352,5164,5252,6026


In [277]:
df1.columns = df1.iloc[0]

# Shifting row 0, the years, to become the index row

df1 = df1.drop(0)
df1=df1.reset_index(drop=True)
del df1['Empty']
df1


,,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Total tax revenue,10945,11946,13320,14976,16934,19080,20167,21474,23324,28900,37343,43954,50389,55380,63624,81153,92880,108233,113345,122092,133701,145729,154902,173088,186990,202368,205304,207654,211438,228041,249597,264778,284587,312365,332778,359690,369143,374624,392640,424991,454976,483578,509308,512205,481292,515253,546130,550431,570196,590196,611338,649143,681497,704795,726723,690450,775646
1,Income tax,3615,4446,4115,4585,5157,5752,6493,6687,7371,9869,14448,16801,17453,18785,20355,23376,26958,29786,31023,32064,33965,39283,40040,43268,46546,57614,58951,61021,60987,65717,71121,71206,72525,85903,93079,101925,107661,108525,110203,117369,127803,136611,147309,149228,138023,144502,149353,146474,152668,156187,163291,169076,177403,184957,191303,186966,218021
2,Employee National Insurance,770,808,854,953,992,1171,1237,1468,1693,1991,2562,3114,3520,3690,4284,5228,6552,8016,9413,10200,11220,12625,12513,13564,14140,13296,13828,14104,14748,17357,18646,19149,21558,22528,23573,24175,25236,25357,29055,32805,35159,37052,36585,38186,37184,38703,40626,41159,41481,42556,44488,47295,50397,52203,56222,56847,61073
3,Self employed National Insurance,75,79,82,94,99,114,120,144,169,193,195,228,277,319,264,317,355,482,552,600,745,725,755,942,1048,1177,1206,1281,1472,1469,1541,1771,1848,1729,1883,2049,2183,2318,2595,2727,2825,2930,2860,3053,2850,2756,2641,2560,2585,2672,3194,3295,3580,3848,4356,4101,4105
4,Capital gains,3,7,22,51,156,264,245,276,377,496,488,420,403,416,529,492,487,592,643,694,855,971,1230,2172,2002,1892,1359,1073,785,821,852,972,482,1460,1971,2211,3400,2820,1526,2296,2332,2874,3848,5288,7815,2517,3600,4334,3902,3914,5582,7060,8560,7797,9187,10253,12109
5,Corporation tax,484,141,1171,1287,1358,1658,1527,1429,1939,2847,2310,2168,3037,3839,4884,6783,8471,10492,12069,13997,16854,15345,16264,18808,23433,20070,16736,12976,12578,15398,20174,24726,32001,34020,32743,38061,34255,31274,31467,38076,47014,50269,50159,45856,38823,45752,45216,43222,43199,45026,43824,54611,54561,54691,51253,50077,61360
6,Employers NI,831,906,976,1101,1136,1354,1453,1708,2054,2791,4068,5066,5694,6069,6889,8210,8814,9344,10536,11269,12245,13540,14395,16176,18145,19984,21159,21621,23038,23240,24210,25579,27200,28731,31286,34028,35706,35735,39890,43692,46475,49568,53765,57080,54411,55887,58174,60600,62019,63892,66491,71373,76472,79327,82728,84042,89367
7,Property taxes,1591,1742,1875,2074,2265,2378,2679,3162,3166,3686,4746,5363,6057,6641,7902,9774,12386,13850,13981,14603,16007,18101,20393,22356,23676,16538,15374,15284,21194,23808,25033,27218,29641,32543,35988,41134,42247,44644,46555,50393,53838,59656,63392,60086,58413,61768,63405,65320,70889,74593,76427,81212,85545,88654,90329,80134,88393
8,Inheritance tax,287,308,317,380,450,383,404,483,419,380,307,390,393,372,418,479,1010,701,690,750,923,1025,1089,1107,1182,1321,1263,1227,1279,1438,1441,1621,1601,1795,1951,2215,2396,2381,2416,2871,3150,3575,3867,3257,2401,2642,2936,3129,4255,3886,4470,4804,5381,5352,5164,5252,6026
9,VAT,0,0,0,0,0,0,0,0,1533,2406,3326,3715,4308,5214,7952,11897,13056,14308,16216,18437,21228,22715,25021,28861,31737,34136,37523,41031,41762,44690,47539,51692,54475,57539,62073,65018,67859,72136,78356,82691,84649,89016,93374,93434,81122,97646,113485,116462,121460,127647,132906,137214,143382,149304,154996,139461,157118


In [278]:
cols_to_convert = df1.columns[1:]

# Converting all the columns, except the names to integers
df1[cols_to_convert] = df1[cols_to_convert].apply(pd.to_numeric, errors='coerce')

df1

,,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Total tax revenue,10945,11946,13320,14976,16934,19080,20167,21474,23324,28900,37343,43954,50389,55380,63624,81153,92880,108233,113345,122092,133701,145729,154902,173088,186990,202368,205304,207654,211438,228041,249597,264778,284587,312365,332778,359690,369143,374624,392640,424991,454976,483578,509308,512205,481292,515253,546130,550431,570196,590196,611338,649143,681497,704795,726723,690450,775646
1,Income tax,3615,4446,4115,4585,5157,5752,6493,6687,7371,9869,14448,16801,17453,18785,20355,23376,26958,29786,31023,32064,33965,39283,40040,43268,46546,57614,58951,61021,60987,65717,71121,71206,72525,85903,93079,101925,107661,108525,110203,117369,127803,136611,147309,149228,138023,144502,149353,146474,152668,156187,163291,169076,177403,184957,191303,186966,218021
2,Employee National Insurance,770,808,854,953,992,1171,1237,1468,1693,1991,2562,3114,3520,3690,4284,5228,6552,8016,9413,10200,11220,12625,12513,13564,14140,13296,13828,14104,14748,17357,18646,19149,21558,22528,23573,24175,25236,25357,29055,32805,35159,37052,36585,38186,37184,38703,40626,41159,41481,42556,44488,47295,50397,52203,56222,56847,61073
3,Self employed National Insurance,75,79,82,94,99,114,120,144,169,193,195,228,277,319,264,317,355,482,552,600,745,725,755,942,1048,1177,1206,1281,1472,1469,1541,1771,1848,1729,1883,2049,2183,2318,2595,2727,2825,2930,2860,3053,2850,2756,2641,2560,2585,2672,3194,3295,3580,3848,4356,4101,4105
4,Capital gains,3,7,22,51,156,264,245,276,377,496,488,420,403,416,529,492,487,592,643,694,855,971,1230,2172,2002,1892,1359,1073,785,821,852,972,482,1460,1971,2211,3400,2820,1526,2296,2332,2874,3848,5288,7815,2517,3600,4334,3902,3914,5582,7060,8560,7797,9187,10253,12109
5,Corporation tax,484,141,1171,1287,1358,1658,1527,1429,1939,2847,2310,2168,3037,3839,4884,6783,8471,10492,12069,13997,16854,15345,16264,18808,23433,20070,16736,12976,12578,15398,20174,24726,32001,34020,32743,38061,34255,31274,31467,38076,47014,50269,50159,45856,38823,45752,45216,43222,43199,45026,43824,54611,54561,54691,51253,50077,61360
6,Employers NI,831,906,976,1101,1136,1354,1453,1708,2054,2791,4068,5066,5694,6069,6889,8210,8814,9344,10536,11269,12245,13540,14395,16176,18145,19984,21159,21621,23038,23240,24210,25579,27200,28731,31286,34028,35706,35735,39890,43692,46475,49568,53765,57080,54411,55887,58174,60600,62019,63892,66491,71373,76472,79327,82728,84042,89367
7,Property taxes,1591,1742,1875,2074,2265,2378,2679,3162,3166,3686,4746,5363,6057,6641,7902,9774,12386,13850,13981,14603,16007,18101,20393,22356,23676,16538,15374,15284,21194,23808,25033,27218,29641,32543,35988,41134,42247,44644,46555,50393,53838,59656,63392,60086,58413,61768,63405,65320,70889,74593,76427,81212,85545,88654,90329,80134,88393
8,Inheritance tax,287,308,317,380,450,383,404,483,419,380,307,390,393,372,418,479,1010,701,690,750,923,1025,1089,1107,1182,1321,1263,1227,1279,1438,1441,1621,1601,1795,1951,2215,2396,2381,2416,2871,3150,3575,3867,3257,2401,2642,2936,3129,4255,3886,4470,4804,5381,5352,5164,5252,6026
9,VAT,0,0,0,0,0,0,0,0,1533,2406,3326,3715,4308,5214,7952,11897,13056,14308,16216,18437,21228,22715,25021,28861,31737,34136,37523,41031,41762,44690,47539,51692,54475,57539,62073,65018,67859,72136,78356,82691,84649,89016,93374,93434,81122,97646,113485,116462,121460,127647,132906,137214,143382,149304,154996,139461,157118


In [279]:
df1 = df1.T
# Transposing the dataframe

df1['Year'] = df1.index

df1.columns = df1.iloc[0]

df1.drop(index=df1.iloc[:1, :].index.tolist(), inplace=True)

df1.reset_index(inplace=True)

In [280]:
df1.drop(0, axis=1, inplace=True)
cols = df1.columns.tolist()

# Rearranging the order
cols = cols[-1:] + cols[:-1]
cols

df1=df1[cols]

# Giving a name to the column with no name
df1.rename(columns={"": "Year"}, inplace=True)
df1.head()


,Year,Total tax revenue,Income tax,Employee National Insurance,Self employed National Insurance,Capital gains,Corporation tax,Employers NI,Property taxes,Inheritance tax,VAT
0,1965,10945,3615,770,75,3,484,831,1591,287,0
1,1966,11946,4446,808,79,7,141,906,1742,308,0
2,1967,13320,4115,854,82,22,1171,976,1875,317,0
3,1968,14976,4585,953,94,51,1287,1101,2074,380,0
4,1969,16934,5157,992,99,156,1358,1136,2265,450,0


In [329]:
from functools import reduce

# Scraping a website contain inflation data

Inflation = pd.read_html(
    'https://www.worlddata.info/europe/united-kingdom/inflation-rates.php')

df_Inflation = Inflation[0]


# Cleaning the data, removing the % signs and making floats
df_Inflation['United Kingdom'] = df_Inflation['United Kingdom'].str.rstrip(
    '%').astype('float')
df_Inflation['Ø EU'] = df_Inflation['Ø EU'].str.rstrip('%').astype('float')
df_Inflation['Ø USA'] = df_Inflation['Ø USA'].str.rstrip(
    '%').astype('float')
df_Inflation['Ø World'] = df_Inflation['Ø World'].str.rstrip(
    '%').astype('float')

df1['Year'] = df1['Year'].astype(int)

# Setting base value to calculate index values
df_Inflation.loc[0, 'UK index'] = 1
df_Inflation.loc[0, 'EU index'] = 1
df_Inflation.loc[0, 'USA index'] = 1


# The data only contained anual growth. I want it to be an index to use convert all values to 2021 equivalent

# I use a loop to perform the formula on each row, row by row.
number = 1
for _ in range(61):
    df_Inflation.loc[(number), 'UK index'] = (df_Inflation.loc[(number-1), 'UK index'])/(1+((df_Inflation.loc[(number-1), 'United Kingdom'])/100))
    number = number + 1


number = 1
for _ in range(61):
    df_Inflation.loc[(number), 'EU index'] = (df_Inflation.loc[(number-1), 'EU index'])/(1+((df_Inflation.loc[(number-1), 'Ø EU'])/100))
    number = number + 1

number = 1
for _ in range(61):
    df_Inflation.loc[(number), 'USA index'] = (df_Inflation.loc[(number-1), 'USA index'])/(1+((df_Inflation.loc[(number-1), 'Ø USA'])/100))
    number = number + 1

df_Inflation.drop(['United Kingdom', 'Ø EU','Ø USA','Ø World'], axis=1, inplace=True)


dfs = [df1, df_Inflation]
merged_df = reduce(lambda left, right: pd.merge(
    left, right, on=['Year'], how='inner'), dfs)


merged_df['Year'] = merged_df['Year'].astype(str)
merged_df.head()


,Year,Total tax revenue,Income tax,Employee National Insurance,Self employed National Insurance,Capital gains,Corporation tax,Employers NI,Property taxes,Inheritance tax,VAT,UK index,EU index,USA index
0,1965,10945,3615,770,75,3,484,831,1591,287,0,0.060267,0.084272,0.116302
1,1966,11946,4446,808,79,7,141,906,1742,308,0,0.062624,0.087390,0.119815
2,1967,13320,4115,854,82,22,1171,976,1875,317,0,0.064177,0.090291,0.123134
3,1968,14976,4585,953,94,51,1287,1101,2074,380,0,0.067193,0.093216,0.128391
4,1969,16934,5157,992,99,156,1358,1136,2265,450,0,0.070855,0.095705,0.135402


In [330]:
# Here I import a csv of UK GDP

df = pd.read_csv("UK GDP data.csv")
df = df.rename(columns={'NGDPRSAXDCGBQ': 'Quarterly GDP'})
df['Year'] = df['DATE'].astype(str).str[:4]
df = df.drop('DATE', axis=1)
df = df.reset_index()

# As the data is quartly, I sum the value of all quarters of same year to get the yearly values
df_GDP = df.groupby("Year")["Quarterly GDP"].sum().reset_index()
df_GDP['Year'] = df_GDP['Year'].astype(str)
df_GDP = df_GDP.rename(columns={'Quarterly GDP': 'Yearly GDP'})

# Merging with the other data
merged_df = pd.merge(merged_df, df_GDP, on='Year', how='inner')
merged_df.head()


,Year,Total tax revenue,Income tax,Employee National Insurance,Self employed National Insurance,Capital gains,Corporation tax,Employers NI,Property taxes,Inheritance tax,VAT,UK index,EU index,USA index,Yearly GDP
0,1965,10945,3615,770,75,3,484,831,1591,287,0,0.060267,0.084272,0.116302,676536.0
1,1966,11946,4446,808,79,7,141,906,1742,308,0,0.062624,0.087390,0.119815,687017.0
2,1967,13320,4115,854,82,22,1171,976,1875,317,0,0.064177,0.090291,0.123134,705998.0
3,1968,14976,4585,953,94,51,1287,1101,2074,380,0,0.067193,0.093216,0.128391,744483.0
4,1969,16934,5157,992,99,156,1358,1136,2265,450,0,0.070855,0.095705,0.135402,758753.0


In [331]:
# Adding new coloums adjusting for inflation
merged_df["Adjusted Total tax revenue"] = merged_df["Total tax revenue"]/merged_df["UK index"]
merged_df["Adjusted Income tax"] = merged_df["Income tax"] / merged_df["UK index"]
merged_df["Adjusted Employee NI"] = merged_df["Employee National Insurance"] / merged_df["UK index"]
merged_df["Adjusted Self employed NI"] = merged_df["Self employed National Insurance"] / merged_df["UK index"]
merged_df["Adjusted Employer NI"] = merged_df["Employers NI"] / merged_df["UK index"]
merged_df["Adjusted Capital gains"] = merged_df["Capital gains"] / merged_df["UK index"]
merged_df["Adjusted Corporation tax"] = merged_df["Corporation tax"] / merged_df["UK index"]
merged_df["Adjusted Property taxes"] = merged_df["Property taxes"] / merged_df["UK index"]
merged_df["Adjusted Inheritance tax"] = merged_df["Inheritance tax"] / merged_df["UK index"]
merged_df["Adjusted VAT"] = merged_df["VAT"] / merged_df["UK index"]

# New column, calculates tax revenue as percentage of GDP 
merged_df["Tax % GDP"] = merged_df["Adjusted Total tax revenue"] / merged_df["Yearly GDP"]

# New column, calculates other from deducting all from total
merged_df["Adjusted Other"] = merged_df["Adjusted Total tax revenue"] - \
    (merged_df["Adjusted Income tax"] + merged_df["Adjusted Employee NI"]  +
     merged_df["Adjusted Self employed NI"] + merged_df["Adjusted Employer NI"] + merged_df["Adjusted Capital gains"] + merged_df["Adjusted Corporation tax"] + merged_df["Adjusted Property taxes"] + merged_df["Adjusted Inheritance tax"] + merged_df["Adjusted VAT"])


# Converting from wide form to long form dataframe
merged_df = pd.melt(merged_df, id_vars='Year', value_vars=['Total tax revenue', 'Income tax', 'Employee National Insurance', 'Self employed National Insurance', 'Employers NI', 'Capital gains',
                    'Corporation tax', 'Property taxes', 'Inheritance tax', 'VAT', 'Yearly GDP', "Adjusted Total tax revenue", "Adjusted Income tax", "Adjusted Employee NI", "Adjusted Self employed NI", "Adjusted Employer NI", "Adjusted Capital gains", "Adjusted Corporation tax", "Adjusted Property taxes", "Adjusted Inheritance tax", "Adjusted VAT", "Tax % GDP", "Adjusted Other"], var_name='Type', value_name='Value')

merged_df


,Year,Type,Value
0,1965,Total tax revenue,10945
1,1966,Total tax revenue,11946
2,1967,Total tax revenue,13320
3,1968,Total tax revenue,14976
4,1969,Total tax revenue,16934
...,...,...,...
1306,2017,Adjusted Other,82121.719422
1307,2018,Adjusted Other,82859.763094
1308,2019,Adjusted Other,84054.847534
1309,2020,Adjusted Other,75164.5884


In [332]:
import json
# Exporting the dataframe as a json file
exported_values = list(merged_df.T.to_dict().values())
open('Tax breakdown.json', 'w').write(json.dumps(exported_values))


91523